In [2]:
import load_dataset
import config
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage
import pandas as pd
import cv2
import numpy as np
import json
import matplotlib.pyplot as plt

annotation_list = load_dataset.cocokeypoints_list_converter(str(config.ANNOTATION_DATA))
training_samples, valid_samples = load_dataset.train_test_split(
    annotation_list,
    config.TEST_SPLIT,
)

Training sample instances: 306
Validation sample instances: 6


In [34]:
# def AugmentFaceKeypointDataset(training_samples, data_path, aug_data_num, mix_augmentent = (3,5)):
# data_path = f"{config.DATASET_PATH}/Anime2Sketch"
data_path = f"{config.DATASET_PATH}/canny_and_dilation"
# data_path = f"{config.DATASET_PATH}/only_canny"

data_set_list = []
resize_seq = iaa.Resize({"height": config.RESIZE, "width": config.RESIZE})
mix_augmentent = (3,6)
aug_data_num = 10
# data_num = 1
print("shape[0]とは",training_samples.shape[0])

# print("len type",type(len(training_samples)))
total_data_count = int(training_samples.shape[0]) * int(aug_data_num)
print("総データ拡張数: ",total_data_count)
for data_num in range(training_samples[0:2].shape[0]):
    
    print(f"{data_path}/{training_samples.iloc[data_num, 0]}")
    image_orig = cv2.imread(f"{data_path}/{training_samples.iloc[data_num, 0]}")
    # image_orig = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    orig_h, orig_w, _ =  image_orig.shape
    # サイズ変更
    image = cv2.resize(image_orig, (512, 512))

    keypoints = training_samples.iloc[data_num][1:]
    keypoints = np.array(keypoints, dtype="float32")
    keypoints = keypoints.reshape(-1, 2)
    keypoints_per = keypoints * [1 / (orig_w), 1 / (orig_h)]


    # print("オリジナル画像")
    if aug_data_num == 0:
        continue

    landmark_num = len(keypoints)
    kps = KeypointsOnImage(
        [Keypoint(x=keypoints[i][0], y=keypoints[i][1]) for i in range(0,landmark_num)],
        shape=image.shape,
    )

    image_resize, kps_resize = resize_seq(image=image, keypoints=kps)
    data_set_list.append([image_resize, keypoints_per])

    # seq = iaa.SomeOf(mix_augmentent, [
    #     iaa.Add((-40, 40), per_channel=0.5),
    #     iaa.AdditiveGaussianNoise(scale=(0, 0.2*255)),
    #     iaa.Cutout(nb_iterations=(10, 70),size=0.05,cval=(0, 255),fill_per_channel=0.5),
    #     iaa.JpegCompression(compression=(80, 99)),
    #     iaa.BlendAlpha([0.25, 0.75], iaa.MedianBlur(13)),

    #     iaa.LogContrast(gain=(0.6, 1.4), per_channel=True),
    #     iaa.Affine(scale={"x": (0.7, 1.1), "y": (0.7, 1.1)}),
    #     iaa.Affine(rotate=(-45, 45)),

    #     iaa.Affine(translate_percent={"x": -0.20}, mode=ia.ALL, cval=(0, 255)),
    #     iaa.ShearX((-20, 20)),
    #     iaa.ShearY((-20, 20)),
    #     iaa.PiecewiseAffine(scale=(0.03, 0.03)),
    #     iaa.imgcorruptlike.Spatter(severity=2),
    #     iaa.Superpixels(p_replace=0.1, n_segments=100)
    # ],random_order=True)
    seq = iaa.SomeOf(mix_augmentent,[
        iaa.AdditiveGaussianNoise(scale=(0, 0.2*255)),
        iaa.Cutout(nb_iterations=(10, 35),size=0.05,cval=(0, 0),fill_per_channel=0.5),
        iaa.Cutout(nb_iterations=(10, 35),size=0.05,cval=(255,255),fill_per_channel=0.5),

        iaa.Affine(scale={"x": (0.7, 1.1), "y": (0.7, 1.1)}),
        iaa.Affine(rotate=(-45, 45)),

        iaa.ShearX((-30, 30)),
        iaa.ShearY((-30, 30)),
        iaa.PiecewiseAffine(scale=(0.03, 0.03))
    ],random_order=True)

    for aug_count in range(aug_data_num-1):
        image_aug, kps_aug = seq(image=image_resize, keypoints=kps_resize)
        keypoints = []
        for i in range(len(kps.keypoints)):
            before = kps.keypoints[i]
            after = kps_aug.keypoints[i]
            keypoints.append([after.x, after.y])
        keypoints = np.array(keypoints, dtype="float32")
        keypoints_per = keypoints * [1 / (orig_w), 1 / (orig_h)]
        image_after = kps_aug.draw_on_image(image_aug, size=0)

        # データ拡張を行った画像をリストに格納する
        data_set_list.append([image_after, keypoints_per])
        total_data_count -= 1
        # plt.imshow(image_aug, cmap='gray')
        # plt.show()
        # plt.close()

        print("残りのデータ拡張数: ",total_data_count)

shape[0]とは 306
総データ拡張数:  3060
/root/dataset/anime_face_landmark_20230912/canny_and_dilation/images/AnythingV5Ink_gitv1.5.4_0901_140034_2.jpg
残りのデータ拡張数:  3059
残りのデータ拡張数:  3058
残りのデータ拡張数:  3057
残りのデータ拡張数:  3056
残りのデータ拡張数:  3055
残りのデータ拡張数:  3054
残りのデータ拡張数:  3053
残りのデータ拡張数:  3052
残りのデータ拡張数:  3051
/root/dataset/anime_face_landmark_20230912/canny_and_dilation/images/AnythingV5Ink_gitv1.5.4_0901_140221_1.jpg
残りのデータ拡張数:  3050
残りのデータ拡張数:  3049
残りのデータ拡張数:  3048
残りのデータ拡張数:  3047
残りのデータ拡張数:  3046
残りのデータ拡張数:  3045
残りのデータ拡張数:  3044
残りのデータ拡張数:  3043
残りのデータ拡張数:  3042


In [35]:
data_set_list

[[array([[[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         ...,
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255